In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

data = pd.read_csv("IMS_Dataset.csv")
label_encoder_customer = LabelEncoder()
label_encoder_customer.fit(data['Customer_ID'])
label_encoder_cluster = LabelEncoder()
label_encoder_cluster.fit(data['Cluster_ID'])
data['Customer_ID_encoded'] = label_encoder_customer.transform(data['Customer_ID'])
data['Cluster_ID_encoded'] = label_encoder_cluster.transform(data['Cluster_ID'])

In [2]:
data

,Indent_ID,Truck_Number,Date_of_Ordering,Customer_ID,Cluster_ID,MS_Requirement,HSD_Requirement,LDO_Requirement,FO_Requirement,LSHS_Requirement,SKO_Requirement,Latitude,Longitude,Truck_Capacity,Customer_ID_encoded,Cluster_ID_encoded
0,INDENT1,Cluster1_Truck14,2022-09-06,Customer1,Cluster1,30.06,14.76,32.28,54.27,7.15,24.04,11.734536,83.450264,530.0,0,0
1,INDENT2001,Cluster1_Truck14,2022-02-10,Customer1,Cluster1,44.19,23.77,18.36,59.92,25.28,12.85,15.915821,71.563455,530.0,0,0
2,INDENT3401,Cluster1_Truck14,2022-03-15,Customer1401,Cluster1,16.77,42.87,38.45,16.95,5.39,3.30,18.785632,87.489529,530.0,448,0
3,INDENT3501,Cluster1_Truck14,2022-08-29,Customer1501,Cluster1,19.61,45.07,2.71,13.93,30.01,15.63,16.858277,83.237239,530.0,559,0
4,INDENT4001,Cluster1_Truck14,2023-10-05,Customer1,Cluster1,45.35,30.94,15.53,32.65,52.22,34.18,27.176815,72.909129,530.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,INDENT17391,Cluster91_Truck1815,2022-08-17,Customer1391,Cluster91,44.55,26.79,1.34,19.97,17.20,51.49,29.722794,80.940190,110.0,436,91
17996,INDENT11716,Cluster16_Truck320,2022-07-19,Customer1716,Cluster16,6.68,11.23,49.84,18.95,30.09,51.25,15.626660,77.026131,160.0,797,8
17997,INDENT12216,Cluster16_Truck320,2023-06-23,Customer216,Cluster16,20.58,41.93,5.53,47.65,52.08,26.83,18.869624,75.455544,160.0,1131,8
17998,INDENT16016,Cluster16_Truck320,2022-01-22,Customer16,Cluster16,44.27,43.87,19.55,11.91,19.37,39.40,15.825518,80.461659,160.0,667,8


In [3]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['MS_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for MS_Requirement : 17.43047804300328


In [4]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['MS_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['HSD_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for HSD_Requirement : 17.54488737308789


In [5]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['MS_Requirement', 'HSD_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['LDO_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for LDO_Requirement : 17.367840803637545


In [6]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['MS_Requirement', 'HSD_Requirement', 'LDO_Requirement', 'LSHS_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['FO_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for FO_Requirement : 17.22227617081089


In [7]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['MS_Requirement', 'HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'SKO_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['LSHS_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for LSHS_Requirement : 17.694538717619203


In [8]:
def train_predict_requirement(requirement_column):
    # Split dataset into features and target variable for the requirement
    X = data[['MS_Requirement', 'HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'Customer_ID_encoded', 'Cluster_ID_encoded']]
    y = data[requirement_column]
    # Split dataset into features and target variable for the requirement
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Train a Random Forest Regression model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    # Predict requirement for test set
    requirement_predicted = model.predict(X_test)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, requirement_predicted))
    print("RMSE for", requirement_column, ":", rmse)
    # Create a DataFrame containing predicted requirement and corresponding Customer_ID, Cluster_ID
    prediction_df = pd.DataFrame({requirement_column + '_Predicted': requirement_predicted, 'Customer_ID': label_encoder_customer.inverse_transform(X_test['Customer_ID_encoded']), 'Cluster_ID': label_encoder_cluster.inverse_transform(X_test['Cluster_ID_encoded'])})
    # Group predictions by Customer_ID and take the mean
    prediction_df = prediction_df.groupby(['Customer_ID', 'Cluster_ID']).mean().reset_index()
    # Output prediction dataset to CSV
    prediction_df.to_csv(requirement_column + "_Predictions.csv", index=False)
# Train and predict for each requirement
for requirement_column in ['SKO_Requirement']:
    train_predict_requirement(requirement_column)

RMSE for SKO_Requirement : 17.69524799589835


In [9]:
# for requirement_column in ['MS_Requirement', 'HSD_Requirement', 'LDO_Requirement', 'FO_Requirement', 'LSHS_Requirement', 'SKO_Requirement']: